In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,29693
2,Huelva,Confirmados PDIA 14 días,1162
3,Huelva,Tasa PDIA 14 días,"226,4356840812986"
4,Huelva,Confirmados PDIA 7 días,617
5,Huelva,Total Confirmados,29869
6,Huelva,Curados,26648
7,Huelva,Fallecidos,374


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  29693.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  7761.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 795 personas en los últimos 7 días 

Un positivo PCR cada 402 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,29693.0,617.0,1162.0,513170.0,120.233061,226.435684,147.0
Condado-Campiña,9518.0,282.0,521.0,156231.0,180.501949,333.480551,74.0
Huelva-Costa,17202.0,281.0,552.0,289548.0,97.047812,190.641966,60.0
Huelva (capital),7761.0,181.0,358.0,143837.0,125.836885,248.892844,39.0
Almonte,1672.0,68.0,161.0,24507.0,277.471743,656.955156,17.0
Bollullos Par del Condado,913.0,50.0,97.0,14387.0,347.535970,674.219782,17.0
Moguer,1169.0,50.0,67.0,21867.0,228.655051,306.397768,14.0
Sierra de Huelva-Andévalo Central,2632.0,54.0,88.0,67391.0,80.129394,130.581235,13.0
Valverde del Camino,511.0,27.0,37.0,12750.0,211.764706,290.196078,10.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Galaroza,41.0,9.0,14.0,1382.0,651.230101,1013.024602,2.0
Bollullos Par del Condado,913.0,50.0,97.0,14387.0,347.535970,674.219782,17.0
Almonte,1672.0,68.0,161.0,24507.0,277.471743,656.955156,17.0
Escacena del Campo,154.0,6.0,10.0,2287.0,262.352427,437.254045,0.0
Palma del Condado (La),1076.0,28.0,39.0,10801.0,259.235256,361.077678,10.0
Condado-Campiña,9518.0,282.0,521.0,156231.0,180.501949,333.480551,74.0
Niebla,207.0,7.0,13.0,4116.0,170.068027,315.840622,2.0
Nerva,169.0,7.0,16.0,5169.0,135.422712,309.537628,0.0
Moguer,1169.0,50.0,67.0,21867.0,228.655051,306.397768,14.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Cartaya,1294.0,5.0,25.0,20083.0,24.896679,124.483394,2.0,0.200000
Aracena,514.0,1.0,5.0,8255.0,12.113870,60.569352,0.0,0.200000
Palos de la Frontera,605.0,7.0,23.0,11742.0,59.615057,195.878045,0.0,0.304348
Beas,204.0,2.0,6.0,4341.0,46.072334,138.217001,1.0,0.333333
Villablanca,185.0,2.0,5.0,2885.0,69.324090,173.310225,1.0,0.400000
Rociana del Condado,534.0,8.0,19.0,7939.0,100.768359,239.324852,0.0,0.421053
Almonte,1672.0,68.0,161.0,24507.0,277.471743,656.955156,17.0,0.422360
Bonares,261.0,3.0,7.0,6060.0,49.504950,115.511551,1.0,0.428571
Nerva,169.0,7.0,16.0,5169.0,135.422712,309.537628,0.0,0.437500
